In [4]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [5]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [6]:
paginas = ['https://icsr.iitpkd.ac.in/past-grant-calls', 'https://icsr.iitpkd.ac.in/grant-calls']
india_IITP = pd.DataFrame()
for pagina in paginas:

    parser_general = parser(pagina)
    try: 
        ult_page = int(parser_general.xpath('//ul//li[@class="pager-last"]/a/@href')[0].split('=')[1])
    except IndexError:
        ult_page = 0

    ##################################
    if pagina == paginas[0]:
        base = 'https://icsr.iitpkd.ac.in/past-grant-calls?page='

        for i in range(ult_page+1):

            page = base+str(i)

            parser_page = parser(page)

            #########################

            titulo = parser_page.xpath('//tr//td[1]/a/text()')

            link = parser_page.xpath('//tr//td[2]/a/@href')

            departamento = parser_page.xpath('//tr//td[2]/a/text()')

            fecha_limite = parser_page.xpath('//tr//td[3]/span/text()')

            fecha_instituto = parser_page.xpath('//tr//td[4]/span/text()')

            india = pd.DataFrame()
            india['Título'] = titulo
            india['Link_proyecto'] = link
            india['Departamento'] = departamento
            india['Fecha Límite'] = fecha_limite
            india['Fecha Limite instituto'] = fecha_instituto

            india_IITP = pd.concat([india_IITP,india])

############################################

    else:
        base = 'https://icsr.iitpkd.ac.in/grant-calls?page='

        for i in range(ult_page+1):

            page = base+str(i)

            parser_page = parser(page)

            #########################

            titulo = parser_page.xpath('//tr//td[2]/a/text()')
            
################
            links = parser_page.xpath('//tr//td[2]/a/@href')
            link = []
            base_link = 'https://icsr.iitpkd.ac.in/
            for l in links:
                link.append(base_link+l)
###################
            departamento = parser_page.xpath('//tr//td[3]/text()')
            depart = []

            for d in departamento:
                d = d.strip()
                depart.append(d)
#############
            fecha_limite = parser_page.xpath('//tr//td[4]/span/text()')
            
#####################
            fecha_instituto = parser_page.xpath('//tr//td[5]/span/text()')
            

            india = pd.DataFrame()
            india['Título'] = titulo
            india['Link_proyecto'] = link
            india['Departamento'] = depart
            india['Fecha Límite'] = fecha_limite
            india['Fecha Limite instituto'] = fecha_instituto

            india_IITP = pd.concat([india_IITP,india])

        india_IITP = india_IITP.reset_index(drop=True)


In [7]:
 india_IITP.to_excel('india_IITP.xlsx')

In [8]:
 india_IITP

,Título,Link_proyecto,Departamento,Fecha Límite,Fecha Limite instituto
0,India- Sweden Collaborative Industrial Researc...,https://dst.gov.in/callforproposals/india-swed...,Department of Science & Technology,20/5/2021,13/5/2021
1,Scientific and Useful Profound Research Advanc...,https://serbonline.in/SERB/Supra,Science and Engineering Research Board,11/5/2021,4/5/2021
2,Digital Transformation for Green Energy Transi...,https://onlinedst.gov.in/Projectproposalformat...,Department of Science & Technology,7/5/2021,3/5/2021
3,India- Israel Industrial R&D and Technological...,https://www.gita.org.in/OnlineRfp/ProgramInfo....,Department of Science & Technology,3/5/2021,26/4/2021
4,ICSSR-MOST (Taiwan) Joint Call for Research Pr...,https://icssr.org/icssr-most-taiwan-joint-call...,Indian Council of Social Science Research (IC...,15/4/2021,8/4/2021
...,...,...,...,...,...
164,Call for proposal for DST-NGP in 'Geospatial ...,https://icsr.iitpkd.ac.in//node/811,Department of Science and Technology,15/6/2021,8/6/2021
165,"SERB, IRHPA call for establishing Centres on A...",https://icsr.iitpkd.ac.in//node/813,Science & Engineering Research Board (SERB),30/6/2021,23/6/2021
166,Collaborative Scientific Research Programme (C...,https://icsr.iitpkd.ac.in//node/771,Indo French Centre for the Promotion of Advanc...,15/7/2021,8/7/2021
167,Intensification of Research in High Priority A...,https://icsr.iitpkd.ac.in//node/788,Science & Engineering Research Board (SERB),15/7/2021,8/7/2021
